<a href="https://colab.research.google.com/github/AHApiko/Simple-HR-chatbot-using-Langchain/blob/main/LangChain_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -qU "langchain[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.7 MB/s eta 0:00:00


In [6]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.1 MB/s eta 0:00:00


In [9]:
!pip install -U langchain langchain-openai


In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [10]:
!pip install langchain openai tiktoken faiss-cpu

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings

In [11]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Load both legal text files
loader_estatuto = TextLoader("/content/estatuto_trabajadores_cleaned.txt", encoding='utf-8')
loader_convenio = TextLoader("/content/convenio_publicidad_cleaned.txt", encoding='utf-8')

docs_estatuto = loader_estatuto.load()
docs_convenio = loader_convenio.load()

# Combine both into a single list
documents = docs_estatuto + docs_convenio

# Split the text for better retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

# Create embeddings and FAISS vectorstore
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(split_docs, embeddings)

# Use retriever for QA
retriever = db.as_retriever()

In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

<ipython-input-12-4aa3445d5960>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(model_name="gpt-3.5-turbo"),


In [13]:
query = "¿Cuántos días de permiso tengo si me caso?"
result = qa_chain({"query": query})

print("Respuesta:", result['result'])

<ipython-input-13-066c800c7ba5>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Respuesta: Tienes derecho a quince días naturales de permiso retribuido en caso de matrimonio.


In [14]:
query = "How many days off do I get if I get married?"
result = qa_chain({"query": query})

print("Respuesta:", result['result'])

Respuesta: You are entitled to fifteen natural days off if you get married.


In [16]:
query = "Is January 25th a holiday for workers in the advertising sector?"

result = qa_chain({"query": query})

print("Respuesta:", result['result'])

Respuesta: Yes, January 25th is considered a professional holiday for workers in the advertising sector. It is a non-working day and is also payable. If January 25th does not fall on a Friday, its celebration will be moved to the first working Friday after that date. Additionally, December 24th and 31st of each year are also considered paid leave days.
